# Fast.ai Part 1
> Notes for lessons 1-3 of Fast.ai and a small project

- toc: true 
- badges: true
- comments: true
- categories: [fastai, project]
- image: images/chart-preview.png

# Lesson Summary

The first three lessons of the Fast.ai course focus on giving you a high level overview of the course, explain the reason for their top down approach, as well as begin to define some terms, and show some examples. The top down approach means they put a focus in the very beginning to demonstrate the capabilities of machine learning programs, and only once an idea is established will they begin to go into details of how it all ends up working and can be implemented. Before they jump into examples though, there is a section in Lesson 1 where they give a brief overview of what and how a machine learning program is trained.

For those who are learning about machine learning and how neural networks work for the first time, there are many explanations online that are more clear than I could describe it, but I think the most illustrative example is this video by 3Blue1Brown:

> youtube: https://www.youtube.com/watch?v=aircAruvnKk


The concepts in the video above are far beyond what is necessary for the first few lessons though. Beyond the history of machine learning, it includes some demos that introduce the Fast.ai library and show how easy it is to train a neural network to recognize the difference between cats and dogs, or to classify different types of bears from data that was obtained through a Bing search.

The lessons do walk you through all the steps of how to set it all up yourself, but when everything is pretyped for you and the only input you need is to shift-enter through it all, it's easy to gloss over it and not learn what any of the parameters are or what is happening, even at a high level. This will be my attempt at implementing a simple image classifier, but first, we'll need to go over some definitions: